In [ ]:
# Enable all CPU cores
from tqdm import tqdm
import pandas as pd
import re
import pandas as pd

# Read the file content
def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r"\s+structure\s+(?P<structure>.+)\s+nat\s+(?P<nat>\d+)\s+nprim\s+(?P<nprim>\d+)\s+maxl\s+(?P<maxl>\d+)\s+max_nnl\s+(?P<max_nnl>\d+)\s+mean_nnl\s+(?P<mean_nnl>\d+)\s+max_nsh\s+(?P<max_nsh>\d+)\s+cpu_time\s+(?P<cpu_time>[\d.]+)\s+gpu_gb_in\s+(?P<gpu_gb_in>[\d.]+)\s+gpu_gb_out\s+(?P<gpu_gb_out>[\d.]+)\s+gpu_gb_total\s+(?P<gpu_gb_total>[\d.]+)\s+gpu_between_atoms\s+(?P<gpu_between_atoms>[\d.]+)\s+gpu_in_atoms\s+(?P<gpu_in_atoms>[\d.]+)\s+gpu_time\s+(?P<gpu_time>[\d.]+)\s+gpu_transfer_time\s+(?P<gpu_transfer_time>[\d.]+)"

  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)

  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
    'nat', 'nprim', 'maxl', 'max_nnl', 'mean_nnl', 'max_nsh',
    'cpu_time', 'gpu_gb_in', 'gpu_gb_out', 'gpu_gb_total',
    'gpu_between_atoms', 'gpu_in_atoms', 'gpu_time', 'gpu_transfer_time'
  ]
  # # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  df = df.sort_values('nat')
  return df

option = 'RTX4090'
!mkdir -p notes/capture/{option}
!mkdir -p notes/data/{option}
!meson test -C build/ hamiltonian --verbose -t 0 > notes/capture/{option}/gpu.txt
df = parse_to_datafame(f"notes/capture/{option}/gpu.txt")
df.to_csv(f"notes/data/{option}/gpu.csv", index=False)
df

,structure,nat,nprim,maxl,max_nnl,mean_nnl,max_nsh,cpu_time,gpu_gb_in,gpu_gb_out,gpu_gb_total,gpu_between_atoms,gpu_in_atoms,gpu_time,gpu_transfer_time
0,GLU,29,6,1,28,14,2,84.487865,0.000453,0.000444,0.000896,11.482048,0.442272,11.924320,1.218752
4,alkane_64,192,6,1,30,20,2,26.773375,0.013026,0.012976,0.026002,1.124256,0.438400,1.562656,9.479776
5,alkane_128,384,6,1,30,21,2,30.221420,0.052005,0.051905,0.103909,1.960640,0.467968,2.428608,38.033791
1,dna,387,6,2,165,69,3,118.418229,0.232068,0.231803,0.463871,10.613664,1.211232,11.824896,186.631943
6,alkane_256,768,6,1,30,21,2,50.012103,0.207819,0.207618,0.415437,3.168256,0.631808,3.800064,155.214844
2,1lyz-lysozyme,1102,6,2,311,99,3,239.951641,1.749900,1.748891,3.498791,26.539007,1.552448,28.091455,1244.322021
3,103l-hydrolase,1404,6,2,311,99,3,339.549190,2.816439,2.815150,5.631589,32.780991,1.694720,34.475712,2008.862305
7,alkane_512,1536,6,1,30,21,2,94.167576,0.830875,0.830472,1.661347,5.540864,1.251168,6.792032,588.715027
8,alkane_1024,3072,6,1,30,21,2,293.517738,3.322695,3.321889,6.644584,10.413344,1.519296,11.932640,2391.341064
9,alkane_2048,6144,6,1,30,21,2,1122.811957,13.289168,13.287555,26.576724,19.293184,1.683456,20.976641,9391.109375


In [3]:
!nproc
!lscpu

128
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   128
  On-line CPU(s) list:    0-127
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7713 64-Core Processor
    CPU family:           25
    Model:                1
    Thread(s) per core:   2
    Core(s) per socket:   64
    Socket(s):            1
    Stepping:             1
    Frequency boost:      enabled
    CPU max MHz:          3720.7029
    CPU min MHz:          1500.0000
    BogoMIPS:             4000.13
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid aperfmperf ra
                          pl 

In [4]:
!nvidia-smi

Sun May 25 15:49:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.144                Driver Version: 570.144        CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:82:00.0 Off |                  Off |
| 40%   30C    P8             11W /  450W |       1MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----